In [1]:
import cometgp_sandbox as cgp
%matplotlib notebook
from ipywidgets import interact

In [2]:
def simulated_data_comparison(gamma, period, amp):
    
    px, py, x_pos, gp1 = cgp.create_data(gamma, period, amp)
    
    def _do_work(n = 10, m = 0):
        x, y, yerr = cgp.sample_data(px, py, x_pos , n, m)
    
        gp2 = cgp.fit_data(x,y,yerr,gamma,period,gp1, print_results = True)
        
        cgp.plotting(x,y,yerr,py,x_pos,gp2)

    return _do_work

#n will only work if m is set equal to 0

In [3]:
vary_nm = simulated_data_comparison(10,1,1)
interact(vary_nm, n=(0, 100,5), m = (0, 10), continuous_update=False)

A Jupyter Widget

<function __main__.simulated_data_comparison.<locals>._do_work>

### To Do
$\bullet$ read more on the george documentation - period confusion

$\bullet$ add all code to github

$\bullet$ read hogg paper

$\bullet$ read whatever Daniela sends


Principle Simulations
    find a better way of simulating models
        synthetic light curves - Bryce will help :)
        then real data
    cadences (LSST and ZTF)
    bootstraping (fake to real)
    compare to other methods that exist
    implement priors
    MCMC - markov chain monte carlo simulations
    
Next Quarter: get 2 simulations as primary examples (densely populated vs sparesly populated (LSST))
              implement priors - write own MCMC sampler (scary)
              
Goal: plot like above with priors and better simulated data and MCMC

Overall goal: Convince asteroid community to use Gaussian processes for future asteroid models
